In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("../data/processed/forbes_definitive.csv", index_col = 0)

In [3]:
df

,id,worth (BUSD),worthChage (millions USD),realTimePosition,name,position,image,area,source,age,gender,country
0,3033,112.0,0.000,1,Jeff Bezos,1,https://specials-images.forbesimg.com/imageser...,Technology,Amazon,55.0,M,United States
1,3381,90.0,-0.001,2,Bill Gates,2,https://specials-images.forbesimg.com/imageser...,Technology,Microsoft,64.0,M,United States
2,2949,84.0,-0.002,3,Warren Buffett,3,https://specials-images.forbesimg.com/imageser...,Finance and Investments,Berkshire Hathaway,89.0,M,United States
3,7380,72.0,0.000,4,Bernard Arnault,4,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,LVMH,70.0,M,France
4,6568,71.0,0.000,5,Mark Zuckerberg,5,https://specials-images.forbesimg.com/imageser...,Technology,Facebook,35.0,M,United States
...,...,...,...,...,...,...,...,...,...,...,...,...
2208,9856,1.0,0.000,2134,Zhao Xiaoqiang,2204,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,"fashion, entertainment",52.0,M,China
2209,5033,1.0,NaN,2134,Zhou Liangzhang,2205,https://specials-images.forbesimg.com/imageser...,Manufacturing,electrical equipment,57.0,M,China
2210,6612,1.0,0.000,1856,Zhu Xingming,2206,https://specials-images.forbesimg.com/imageser...,Manufacturing,electrical equipment,52.0,M,China
2211,8969,1.0,0.000,1978,Zhuo Jun,2207,https://specials-images.forbesimg.com/imageser...,Manufacturing,printed circuit boards,54.0,F,Hong Kong


In [88]:
countries = list(set(df.country.to_list()))


In [89]:
countries = list(map(lambda x: x.lower().replace(' ', '-') if type(x) == str else None, countries))

In [200]:
countries

[None,
 'morocco',
 'canada',
 'monaco',
 'swaziland',
 'hong-kong',
 'netherlands',
 'hungary',
 'ireland',
 'slovakia',
 'united-kingdom',
 'sweden',
 'norway',
 'peru',
 'united-arab-emirates',
 'switzerland',
 'japan',
 'france',
 'greece',
 'china',
 'russia',
 'guernsey',
 'india',
 'austria',
 'georgia',
 'st.-kitts-and-nevis',
 'israel',
 'germany',
 'denmark',
 'romania',
 'venezuela',
 'united-states',
 'tanzania',
 'poland',
 'singapore',
 'argentina',
 'new-zealand',
 'iceland',
 'qatar',
 'south-africa',
 'indonesia',
 'algeria',
 'kazakhstan',
 'egypt',
 'macau',
 'chile',
 'brazil',
 'nigeria',
 'lebanon',
 'portugal',
 'colombia',
 'nepal',
 'spain',
 'belgium',
 'liechtenstein',
 'vietnam',
 'zimbabwe',
 'south-korea',
 'philippines',
 'turkey',
 'kuwait',
 'taiwan',
 'italy',
 'malaysia',
 'cyprus',
 'australia',
 'ukraine',
 'finland',
 'thailand',
 'mexico',
 'oman',
 'czech-republic']

In [90]:
url = "https://countryeconomy.com/countries/"

In [183]:
df1 = pd.read_html(url+countries[1], attrs = {'class': 'table tabledat table-striped table-condensed table-hover'})[0]

In [184]:
df1.drop(1, axis = 1,  inplace=True)

In [185]:
df1 = df1.T

In [186]:
df1.columns = df1.iloc[0]

In [187]:
df1["Country"] = countries[1]

In [188]:
df1 = df1.drop(0).drop("Gouvernement", axis = 1)

In [189]:
df1

,Annual GDP [+],Annual GDP [+],GDP per capita [+],GDP per capita [+],Debt [+],Debt [+],Debt (%GDP) [+],Debt Per Capita [+],Debt Per Capita [+],Deficit (M.€) [+],...,Population [+],Immigrant stock [+],Emigrant stock [+],HDI [+],Gender Gap Ranking [+],Life expectancy [+],Number of homicides [+],Energy and Environment,CO2 Tons per capita [+],Country
2,"100,414M.€","118,534M.$","2,851€","3,366$",65169,76964,64.96%,"1,850€","2,185$",-3729,...,35220000,95835,2898721,0.667,137º,76.06,431,Energy and Environment,0.24,morocco


In [196]:
df1.loc[:, ["Annual GDP [+]", "GDP per capita [+]"]].iloc[:, [1, 3]]

,Annual GDP [+],GDP per capita [+]
2,"118,534M.$","3,366$"


In [197]:
def web_scrap_economic(url, country):
    df1 = pd.read_html(url + country, attrs = {'class': 'table tabledat table-striped table-condensed table-hover'})[0]
    df1.drop(1, axis = 1,  inplace=True)
    df1 = df1.T
    df1.columns = df1.iloc[0]
    df1 = df1.drop(0).drop("Gouvernement", axis = 1)
    return_df = df1.loc[:, ["Annual GDP [+]", "GDP per capita [+]"]].iloc[:, [1, 3]]
    return return_df

In [206]:
countries_df = pd.DataFrame()
country_col = []
for i in range(1, len(countries)):
    try: 
        df1 = web_scrap_economic(url, countries[i])
    except:
        continue
    country_col.append(countries[i])
    countries_df = pd.concat([countries_df, df1], sort = False)
countries_df["Country"] = country_col
countries_df

,Annual GDP [+],GDP per capita [+],Country
2,"118,534M.$","3,366$",morocco
2,"1,712,479M.$","46,290$",canada
2,"6,401M.$","167,125$",monaco
2,"4,704M.$","4,140$",swaziland
2,"362,721M.$","48,451$",hong-kong
...,...,...,...
2,"274,210M.$","49,738$",finland
2,"504,928M.$","7,448$",thailand
2,"1,222,053M.$","9,797$",mexico
2,"79,277M.$","18,970$",oman


In [207]:
countries_df["Country"] = countries_df.Country.str.replace("-", " ").str.title()

In [208]:
countries_df

,Annual GDP [+],GDP per capita [+],Country
2,"118,534M.$","3,366$",Morocco
2,"1,712,479M.$","46,290$",Canada
2,"6,401M.$","167,125$",Monaco
2,"4,704M.$","4,140$",Swaziland
2,"362,721M.$","48,451$",Hong Kong
...,...,...,...
2,"274,210M.$","49,738$",Finland
2,"504,928M.$","7,448$",Thailand
2,"1,222,053M.$","9,797$",Mexico
2,"79,277M.$","18,970$",Oman


In [209]:
countries_df.to_csv("../data/processed/economic_values.csv")